# Import Packages

In [1]:
from strava_api import StravaAPI
from garmin_file_manager import CleanData,WrangleData, GarminDataManager
from statistics import Statistics
import pandas as pd
import time
import config

%load_ext autoreload
%autoreload 2

## Load Strava Data

In [2]:
# init the api with the access token
strava = StravaAPI()

strava_data_pulls = []
# through trial and error I know I have about 8 pages of data. We could probably automate this but for now this is fine
page_nums = 8
# iterate through each page on strava and request the data
for page_num in range(page_nums):
    strava_data_pulls.append(pd.json_normalize(strava.get_dataset(page_num = page_num + 1)))
    # I don't want to overload my API limits, so I'll give it some time in between requests
    time.sleep(.5)
strava_data = pd.concat(strava_data_pulls, ignore_index = True)

# add to one of the .py at some point
strava_data['datetime'] = pd.to_datetime(strava_data['start_date'])
strava_data['date'] = strava_data['datetime'].dt.date
strava_data['year'] = strava_data['datetime'].dt.year
strava_data['month'] = strava_data['datetime'].dt.month
strava_data['hour'] = strava_data['datetime'].dt.hour

# init the statistics class for our str
ss_strava = Statistics(strava_data)



Requesting Token...



In [49]:
ss_strava.get_max_vals(return_only_max_vals = True)
ss_strava.get_mean_vals()
ss_strava.get_std_vals()
ss_strava.get_min_vals()
ss_strava.get_max_vals(return_only_max_vals = False)#['comment_count']


,Max Value Type,Max Value,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,type,sport_type,...,total_photo_count,has_kudoed,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_watts,kilojoules,device_watts
0,resource_state,2,2,Thinking about making some changes,1503.0,495,514,68.0,Run,Run,...,1,False,42779025,1,a10572939478,kupsF|fbaS\?RBNBLFALYd@k@n@Mb@ATVj@Nf@HbABbBKj...,2,NaN,NaN,NaN
1101,distance,93824,2,New shoes,93824.9,6000,6000,0.0,EBikeRide,EBikeRide,...,0,False,42779025,1,a3072204761,,2,NaN,NaN,NaN
1219,moving_time,32708,2,Heart rate data,0.0,32708,32708,0.0,Run,Run,...,0,False,42779025,1,a2888434025,,2,NaN,NaN,NaN
1219,elapsed_time,32708,2,Heart rate data,0.0,32708,32708,0.0,Run,Run,...,0,False,42779025,1,a2888434025,,2,NaN,NaN,NaN
145,total_elevation_gain,890,2,Run/Swim/Run,1775.8,837,837,890.0,Workout,Workout,...,1,False,42779025,1,a7517449121,gaf|FzxbrLLIXIN?h@IB@DILCCABAJBMNACFMDCHMAIGET...,2,NaN,NaN,NaN
37,workout_type,10,2,Rolling,9837.2,1887,2016,119.0,Ride,Ride,...,1,False,42779025,1,a9613046121,ytpsF|ebaSDJDBn@TBJCLOZm@bAIA_@_@e@_@k@SSCkABs...,2,90.7,171.2,False
0,id,-2147483648,2,Thinking about making some changes,1503.0,495,514,68.0,Run,Run,...,1,False,42779025,1,a10572939478,kupsF|fbaS\?RBNBLFALYd@k@n@Mb@ATVj@Nf@HbABbBKj...,2,NaN,NaN,NaN
130,utc_offset,-14400,2,"Not Ed, Not Edd, just Eddie",5130.0,1573,1596,0.0,Run,Run,...,0,False,42779025,1,a7586188107,,2,NaN,NaN,NaN
567,achievement_count,31,2,Morning Run,24155.4,5863,5863,182.1,Run,Run,...,0,False,42779025,1,a4987942740,a`p}FfncxOGm@Do@yAi@]c@oBuEKeBq@mCf@wCz@aAfASZ...,2,NaN,NaN,NaN
487,kudos_count,33,2,Baptismal Waters 💦,3000.0,631,631,0.0,Run,Run,...,1,False,42779025,1,a5220367137,,2,NaN,NaN,NaN


## Load Garmin Data
There are two ways to get your activities from Garmin. The first is signing into your Garmin connect account, clicking your activities on the sidebar, scrolling down for all the activities that you want to select, then clicking export data in the top right corner. The other involves plugging your watch into your computer, copying and pasting the FIT files, then converting them to excel files with Garming SDK. The first option is easiest in my opinion

In [4]:
# read in the data from the csv files in data
# we might need to read every singele file in data then concat them?
file_path = 'data/' 
garmin_data = pd.read_csv(file_path + 'Activities_miles_mena.csv')
# clean the data
garmin_data_clean = CleanData(garmin_data).data
# wrangle the data
garmin_data_wrangle = WrangleData(garmin_data_clean).data
# init the statistics class for garmin data
ss_garmin = Statistics(garmin_data_wrangle)

## Simple Statistics

In [21]:
#ss_garmin.get_max_vals(return_only_max_vals = True)
#ss_garmin.max_vals(return_only_max_vals = False)
#ss_garmin.get_mean_vals()
#ss_garmin.get_std_vals()
#ss_garmin.get_min_vals()
#ss_garmin.get_median_vals()
#ss_garmin.get_kurtosis_vals()
#ss_garmin.get_correlation()
ss_garmin.get_autocorrelation(7)

{'Distance': 0.22315825105286594,
 'Calories': 0.24971822714088882,
 'Time': 0.11500725767111543,
 'Avg HR': 0.6542290096568928,
 'Max HR': 0.6614877995136339,
 'Avg Pace': 0.2621091318079769,
 'Best Pace': 0.057526430071911126,
 'Total Ascent': 0.15203061110669872,
 'Total Descent': 0.14149149527708807,
 'Avg Stride Length': -0.009557423445734486,
 'Avg Vertical Ratio': nan,
 'Avg Vertical Oscillation': nan,
 'Avg Ground Contact Time': nan,
 'Training Stress Score®': nan,
 'Avg Power': nan,
 'Max Power': nan,
 'Grit': nan,
 'Flow': nan,
 'Avg. Swolf': nan,
 'Avg Stroke Rate': nan,
 'Min Temp': 0.9411359474718929,
 'Best Lap Time': 0.2762855889836433,
 'Number of Laps': 0.22351169735179455,
 'Max Temp': 0.950056552527508,
 'Moving Time': 0.1650180796573693,
 'Elapsed Time': 0.11718529491496597,
 'Min Elevation': 0.7036676705868693,
 'Max Elevation': 0.6788005843868362,
 'Hour': 0.23689175801281992,
 'Minute': 0.008015708110820008,
 'Weekday': -0.1321621210475999}

In [166]:
garmin_data_wrangle.columns#.select_dtypes(include = np.number)#['Distance'].autocorr(7) # method{‘pearson’, ‘kendall’, ‘spearman’}

Index(['Activity Type', 'Date', 'Favorite', 'Title', 'Distance', 'Calories',
       'Time', 'Avg HR', 'Max HR', 'Aerobic TE', 'Avg Run Cadence',
       'Max Run Cadence', 'Avg Pace', 'Best Pace', 'Total Ascent',
       'Total Descent', 'Avg Stride Length', 'Avg Vertical Ratio',
       'Avg Vertical Oscillation', 'Avg Ground Contact Time',
       'Training Stress Score®', 'Avg Power', 'Max Power', 'Grit', 'Flow',
       'Avg. Swolf', 'Avg Stroke Rate', 'Total Reps', 'Dive Time', 'Min Temp',
       'Surface Interval', 'Decompression', 'Best Lap Time', 'Number of Laps',
       'Max Temp', 'Moving Time', 'Elapsed Time', 'Min Elevation',
       'Max Elevation', 'Day', 'Hour', 'Minute', 'Weekday', 'Calendar Date'],
      dtype='object')

## Plotting

In [5]:
import plotly.express as px
import chart_studio.plotly as py

# Run this once. It connects our free chart studio account with our code
#import chart_studio
#chart_studio.tools.set_credentials_file(username=config.plotly_username, api_key=config.plotly_api_key)

In [44]:
ss_garmin.get_numeric()

,Distance,Calories,Time,Avg HR,Max HR,Avg Pace,Best Pace,Total Ascent,Total Descent,Avg Stride Length,...,Best Lap Time,Number of Laps,Max Temp,Moving Time,Elapsed Time,Min Elevation,Max Elevation,Hour,Minute,Weekday
0,0.93,103,8.0,142,165,550,462,223,75,1.16,...,514.17,1,78.8,500.0,8.0,5183,5385,16,28,1
1,1.01,95,8.0,136,151,513,401,194,144,1.28,...,13.73,2,82.4,496.0,9.0,5134,5284,8,15,4
2,0.50,45,4.0,129,143,543,486,89,3,1.16,...,273.55,1,86.0,4.0,4.0,5134,5228,6,18,2
3,1.00,177,2388.0,98,137,2384,912,72,69,0.90,...,3.34,2,73.4,1490.0,2388.0,352,404,11,3,5
4,0.35,39,3.0,139,148,584,497,121,0,1.12,...,202.67,1,82.4,199.0,3.0,5047,5160,16,51,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,5.76,888,2703.0,0,0,469,421,223,241,1.31,...,375.32,6,0.0,2677.0,2838.0,5692,5886,8,52,2
995,7.86,1214,3601.0,0,0,458,382,199,204,1.33,...,408.88,8,0.0,3573.0,3683.0,5623,5739,7,31,6
996,7.76,1199,3602.0,0,0,464,394,125,124,1.33,...,348.22,8,0.0,3571.0,3970.0,5551,5648,8,3,5
997,8.14,1251,3602.0,0,0,443,295,204,206,1.37,...,58.57,9,0.0,3578.0,3646.0,5617,5740,7,22,4


### Running Metrics Over Time

In [41]:
# Running Metrics Over Time
activity_type,var = 'Running', 'Distance'
numeric_cols = ss_garmin.get_numeric().columns

df = garmin_data_wrangle[garmin_data_wrangle['Activity Type'] == 'Running']#.drop('Date', axis = 1).groupby('Calendar Date', as_index=False).sum()

end_range = df[df['Calendar Date'] == '2021-11-29'].index[0]

#fig = px.line(df.iloc[end_range:], x = 'Calendar Date', y = var, title = '%s %s over Time'%(activity_type,var)) #, color = 'Weekday')
fig = px.line(df, x = 'Calendar Date', y = var, title = '%s Metrics over Time'%(activity_type)) #, color = 'Weekday')

#fig.update_layout(margin=dict(l=0, r=0, t=40, b=0),width = 1100, height=400 )
args = [{'x': [df['Calendar Date']], 'y': [df['Distance']]}]

buttons = []
for column in ss_garmin.get_numeric().columns:
    button = dict(
        label=column,
        method='update',
        args=[{'x': [df['Calendar Date']], 'y': [df[column]]}]
    )
    buttons.append(button)
    
fig.update_layout(margin=dict(l=0, r=0, t=40, b=0),width = 1000, height=400,
                 updatemenus= [
            dict(
                type = 'dropdown',
                x = -.02,
                y = 1,
                buttons= buttons
                 
                 )]) 

py.plot(fig, filename = 'basic-line-distance-over-time', auto_open=False)
#fig.show()

'https://plotly.com/~miles_mena5280/1/'

In [52]:
#strava_data

strava_numeric.columns

strava_cols = ['name','achievement_count', 'kudos_count', 'comment_count', 'athlete_count','pr_count', 'total_photo_count']

Index(['resource_state', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'workout_type', 'id', 'utc_offset',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'average_speed', 'max_speed', 'average_cadence',
       'average_temp', 'average_heartrate', 'max_heartrate', 'elev_high',
       'elev_low', 'upload_id', 'pr_count', 'total_photo_count', 'athlete.id',
       'athlete.resource_state', 'map.resource_state', 'average_watts',
       'kilojoules'],
      dtype='object')

In [7]:
strava_data.columns

Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility',
       'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'average_cadence', 'average_temp', 'has_heartrate',
       'average_heartrate', 'max_heartrate', 'heartrate_opt_out',
       'display_hide_heartrate_option', 'elev_high', 'elev_low', 'upload_id',
       'upload_id_str', 'external_id', 'from_accepted_tag', 'pr_count',
       'total_photo_count', 'has_kudoed', 'athlete.id',
       'athlete.resource_state', 'map.id', 'map.summary_polyline',
       'map.resource_state', 'average_watts', 'kilojoules', 'device_watts'],


In [18]:
strava_data['sport_type']

0        Run
1        Run
2        Run
3       Swim
4        Run
        ... 
1433     Run
1434     Run
1435     Run
1436     Run
1437     Run
Name: sport_type, Length: 1438, dtype: object

SyntaxError: invalid syntax (631385301.py, line 8)

In [148]:
# 

#strava_corr_col = ['moving_time','distance','comment_count','photo_count','total_elevation_gain','utc_offset','max_speed', 'killojoules' ]
scatter_cols = ['distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'achievement_count',
        'comment_count', 'athlete_count',
        'average_temp', 'average_heartrate',
        'max_heartrate', 'elev_high',
        'pr_count', 'total_photo_count',
       'elev_low']

var, activity_type = strava_corr_col[1], 'Run'

#strava_numeric = ss_strava.get_numeric()
#strava_numeric.corr()['kudos_count']

df = strava_data[strava_data['sport_type'] == activity_type]

fig = px.scatter(df,
                 y = 'kudos_count',
                 x = 'distance',
                 color =  'total_elevation_gain' ,
                 hover_name = 'name', 
        
                ) 

x_buttons = []
for column in scatter_cols: #ss_strava.get_numeric().columns:
    button = dict(
        label=column,
        method='update',
        args=[{ 'x': [df[column]]},
              {'xaxis': {'title': column}}]
    )
    x_buttons.append(button)

color_buttons = []
for column in scatter_cols: #ss_strava.get_numeric().columns:
    button = dict(
        label=column,
        method='update',
        args=[
            {'marker.color': [df[column]], 'coloraxis.colorscale': 'Viridis'},
            {'coloraxis': {'cmin': df[column].min(), 'cmax': df[column].max(),'colorbar': {'title': {'text': column}}}}
        ]
    )
    color_buttons.append(button)

size_buttons = []
for column in scatter_cols: #ss_strava.get_numeric().columns:
    button = dict(
        label=column,
        method='update',
        args=[
            {'marker.size': [df[column]], 'marker.sizemode': 'area', 'marker.sizeref': df[column].max() / 1000 },
        ]
    )
    size_buttons.append(button)

 
    

fig.update_layout(margin=dict(l=0, r=0, t=40, b=0),width = 800, height=400,
            title=dict(text="Factors in Kudos ", x=0.5, y=0.95, xanchor="center", yanchor="top"),
                  
                 updatemenus= [
            dict(
                type = 'dropdown',
                x = -.15,
                y = .99,
                buttons= x_buttons 
                 ),
            dict(
                type = 'dropdown',
                x = -.16,
                y = .79,
                buttons= color_buttons 
                 ),
            dict(
                type = 'dropdown',
                x = -.15,
                y = .59,
                buttons= size_buttons 
                 )],
                  
    annotations=[
        dict(
            text="Select X-Axis:",
            showarrow=False,
            x=-.5,
            y=1,
            xref="paper",
            yref="paper",
            xanchor="left",
            yanchor="bottom",
            font=dict(size=14)
        ),
        dict(
            text="Select Color:",
            showarrow=False,
            x=-.5,
            y=.8,
            xref="paper",
            yref="paper",
            xanchor="left",
            yanchor="bottom",
            font=dict(size=14)
        ),
        dict(
            text="Select Size:",
            showarrow=False,
            x=-.5,
            y=.6,
            xref="paper",
            yref="paper",
            xanchor="left",
            yanchor="bottom",
            font=dict(size=14)
        )
    ]
                 ) 

py.plot(fig, filename = 'scatter-plot-kudos-factors', auto_open=False)

#fig.show()

'https://plotly.com/~miles_mena5280/26/'

In [146]:
#strava_data.dtypes

#strava_numeric = ss_strava.get_numeric()

#strava_numeric.columns[strava_numeric.eq(0).all()]
ss_strava.get_numeric().columns

scatter_cols = ['distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'achievement_count',
        'comment_count', 'athlete_count',
        'average_temp', 'average_heartrate',
        'max_heartrate', 'elev_high',
        'pr_count', 'total_photo_count',
       'elev_low']

Index(['resource_state', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'workout_type', 'id', 'utc_offset',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'average_speed', 'max_speed', 'average_cadence',
       'average_temp', 'average_heartrate', 'max_heartrate', 'elev_high',
       'elev_low', 'upload_id', 'pr_count', 'total_photo_count', 'athlete.id',
       'athlete.resource_state', 'map.resource_state', 'average_watts',
       'kilojoules'],
      dtype='object')

In [20]:
strava_data.dtypes

resource_state                     int64
name                              object
distance                         float64
moving_time                        int64
elapsed_time                       int64
total_elevation_gain             float64
type                              object
sport_type                        object
workout_type                     float64
id                                 int64
start_date                        object
start_date_local                  object
timezone                          object
utc_offset                       float64
location_city                     object
location_state                    object
location_country                  object
achievement_count                  int64
kudos_count                        int64
comment_count                      int64
athlete_count                      int64
photo_count                        int64
trainer                             bool
commute                             bool
manual          